In [2]:
import pandas as pd
import os
os.chdir('/home/tapas/')
df = pd.read_csv("cs-training.csv")
df.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [9]:
df.number_of_dependents = df.number_of_dependents.fillna(0)
print_null_freq(df)

value,False,True
variable,,
age,150000,0
debt_ratio,150000,0
monthly_income,120269,29731
number_of_dependents,150000,0
number_of_open_credit_lines_and_loans,150000,0
number_of_time30-59_days_past_due_not_worse,150000,0
number_of_time60-89_days_past_due_not_worse,150000,0
number_of_times90_days_late,150000,0
number_real_estate_loans_or_lines,150000,0


Taking a look at monthly_income we see that it's a bit more complicated than number_of_dependents. 
We have a few options for replacing missing data. We could do something like set it to the mean or
median or the dataset but this might skew our distribution. We could also set it to 0 but this might
not be right either. Instead we're going to use a technique called imputation.

In [10]:
df.number_of_dependents.unique()

array([  2.,   1.,   0.,   3.,   4.,   5.,   6.,   8.,   7.,  20.,  10.,
         9.,  13.])

In [11]:
df.to_csv("credit-data-post-import.csv", index=False)

# Imputation

In [12]:
of = pd.read_csv("credit-data-post-import.csv")
df = of[collist]

# Cross Validation

In [14]:
import numpy as np
is_test = np.random.uniform(0, 1, len(df)) > 0.75
train = df[is_test==False]
test = df[is_test==True]

In [15]:
from sklearn.neighbors import KNeighborsRegressor

income_imputer = KNeighborsRegressor(n_neighbors=1)

# split our data into 2 groups; data containing nulls and data 
# not containing nulls we'll train on the latter and make
# 'predictions' on the null data to impute monthly_income
train_w_monthly_income = train[train.monthly_income.isnull()==False]
train_w_null_monthly_income = train[train.monthly_income.isnull()==True]
train_w_monthly_income.corr().ix[:,5]

/root/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


serious_dlqin2yrs                             -0.018108
revolving_utilization_of_unsecured_lines       0.004754
age                                            0.035032
number_of_time30-59_days_past_due_not_worse   -0.008875
debt_ratio                                    -0.029139
monthly_income                                 1.000000
number_of_open_credit_lines_and_loans          0.087799
number_of_times90_days_late                   -0.011589
number_real_estate_loans_or_lines              0.119666
number_of_time60-89_days_past_due_not_worse   -0.009992
number_of_dependents                           0.060984
Name: monthly_income, dtype: float64

In [16]:
cols = ['number_real_estate_loans_or_lines', 'number_of_open_credit_lines_and_loans']
income_imputer.fit(train_w_monthly_income[cols], train_w_monthly_income.monthly_income)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=1, p=2,
          weights='uniform')

In [17]:
new_values = income_imputer.predict(train_w_null_monthly_income[cols])
train_w_null_monthly_income['monthly_income'] = new_values
new_values

/root/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


array([ 1725.,  5510.,  2500., ...,   833.,  2400.,   833.])

In [18]:
#combine the data back together
train = train_w_monthly_income.append(train_w_null_monthly_income)

In [19]:
test['monthly_income_imputed'] = income_imputer.predict(test[cols])

/root/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
test['monthly_income'] = np.where(test.monthly_income.isnull(), test.monthly_income_imputed,
                                  test.monthly_income)

/root/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
train.to_csv("cs-training-missing.csv", index=False)
test.to_csv("cs-test-missing.csv", index=False)